<a href="https://colab.research.google.com/github/realtechsupport/cocktail/blob/main/sandbox/multi_image_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rasterio
!pip install gdal


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [16]:
import os
import numpy as np
import rasterio
from osgeo import gdal, osr
import keras
import tensorflow as tf


In [4]:
datapath = '/content/gdrive/MyDrive/exp/'
ps = 'crop_mask.tif'
patch_size = 256
roipath = '/content/gdrive/MyDrive/exp/other images/roi_folder/'
roishape = 'area2_square.geojson'

In [24]:
from tensorflow.keras.utils import to_categorical
def onehotencoding(labels, num_classes=23):
  return to_categorical(labels, num_classes)

In [5]:
#target clipping

rasterfile = gdal.Open(datapath + ps)

print('\nPerforming the clip operation...\n')


warp_options = gdal.WarpOptions(cutlineDSName = roipath + roishape, cropToCutline = True)
rasterfile_new = ps.split('.tif')[0] + '_roi.tif'
print(roipath + rasterfile_new, datapath + ps )
ds = gdal.Warp(roipath + rasterfile_new, datapath + ps,  options = warp_options)

cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount
projInfo = ds.GetProjection()
spatialRef = osr.SpatialReference()
spatialRef.ImportFromWkt(projInfo)
spatialRefProj = spatialRef.ExportToProj4()
ds = None

print('\nClipped raster input: ', rasterfile_new)
print('Checking spatial reference info\n')
print ("WKT format: " + str(spatialRef))
print ("Proj4 format: " + str(spatialRefProj))
print ("Number of columns: " + str(cols))
print ("Number of rows: " + str(rows))
print ("Number of bands: " + str(bands))


Performing the clip operation...

/content/gdrive/MyDrive/exp/other images/roi_folder/crop_mask_roi.tif /content/gdrive/MyDrive/exp/crop_mask.tif

Clipped raster input:  crop_mask_roi.tif
Checking spatial reference info

WKT format: PROJCS["WGS 84 / UTM zone 50S",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",117],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",10000000],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","

1. Create target pipeline

*   Get the target image and apply preprocessing and create patches
*   apply sampling function and return only useful masks and their indexes



In [28]:
def target_preprocessing(folder_path, patch_size):




    with rasterio.open(folder_path) as src:
        # Read the TIFF data
        output_mask = src.read()

        # Get the shape of the TIFF data
        num_bands, height, width = output_mask.shape

        # Calculate the new width and height that are multiples of the patch size
        new_width = int(np.floor(width / patch_size)) * patch_size
        new_height = int(np.floor(height / patch_size)) * patch_size

        print("cropped dimensions:", new_height, new_width)

        output_mask = np.moveaxis(output_mask, 0, -1)

        # Crop the input_image to the new dimensions
        cropped_mask = output_mask[:new_height, :new_width, :]

    new_mask = np.squeeze(cropped_mask)

    masks = []
    for i in range(0, new_mask.shape[0], patch_size):
        for j in range(0, new_mask.shape[1], patch_size):
            patch = new_mask[i:i+patch_size, j:j+patch_size]
            masks.append(patch)

    mask_array = np.array(masks)

    useful_masks = []
    useless = 0
    indexes = []
    for index,img in enumerate(mask_array):

        val, counts = np.unique(img, return_counts=True)

        if (1 - (counts[0]/counts.sum())) > 0.05:
          useful_masks.append(img)
          indexes.append(index)
        else:
          #print("I am useless")
          useless +=1


    print("Total useful images are: ", len(mask_array)-useless)
    print(indexes)
    print("Total useless images are: ", useless)

    useful_masks = np.array(useful_masks)

    useful_masks_onehot = to_categorical(useful_masks)






    return useful_masks_onehot, indexes






Test the target_preprocessing function: see if it is returning useful_masks
and indexes

In [29]:
folder_path = '/content/gdrive/MyDrive/exp/other images/roi_folder/crop_mask_roi.tif'
patch_size = 256
target = target_preprocessing(folder_path,patch_size)

cropped dimensions: 4096 4608
Total useful images are:  29
[88, 89, 103, 104, 106, 110, 121, 122, 123, 124, 125, 127, 129, 138, 139, 141, 142, 143, 156, 160, 161, 179, 194, 197, 212, 247, 265, 283, 284]
Total useless images are:  259


In [30]:
(target[0][0]).shape

(256, 256, 21)

In [9]:
target[1]
len(target[1])

29

1. Create a preprocessing loop for a folder containing images


*   Each image needs to be clipped, processed, sampled

*   the processed images should go into a list, and correspondingly a list should be filled up with target equivalent to the number of images


In [10]:

def newclipping(datapath,roipath,ps, roishape):
  rasterfile = gdal.Open(datapath + ps)

  print('\nPerforming the clip operation...\n')


  warp_options = gdal.WarpOptions(cutlineDSName = roipath + roishape, cropToCutline = True)
  rasterfile_new = ps.split('.tif')[0] + '_roi.tif'
  print(roipath + rasterfile_new, datapath + ps )
  ds = gdal.Warp(roipath + rasterfile_new, datapath + ps,  options = warp_options)

  cols = ds.RasterXSize
  rows = ds.RasterYSize
  bands = ds.RasterCount
  projInfo = ds.GetProjection()
  spatialRef = osr.SpatialReference()
  spatialRef.ImportFromWkt(projInfo)
  spatialRefProj = spatialRef.ExportToProj4()
  ds = None

  print('\nClipped raster input: ', rasterfile_new)
  print('Checking spatial reference info\n')
  print ("WKT format: " + str(spatialRef))
  print ("Proj4 format: " + str(spatialRefProj))
  print ("Number of columns: " + str(cols))
  print ("Number of rows: " + str(rows))
  print ("Number of bands: " + str(bands))

  return roipath + rasterfile_new



In [11]:
def preprocessing(filelocation,patch_size):
    # Load the GeoTIFF file
    with rasterio.open(filelocation) as src:
        # Read the TIFF data
        tiff_data = src.read()
        print("total number of nan in original",np.count_nonzero(np.isnan(tiff_data)))

        # Get the shape of the TIFF data
        num_bands, height, width = tiff_data.shape

        print("Original image dimensions:", num_bands, height, width)
        unique_elements, counts_elements = np.unique(tiff_data, return_counts=True)
        print(unique_elements, counts_elements )
        print("total unique",len(counts_elements))

        print(np.min(tiff_data), np.max(tiff_data))


        normalized_image = np.zeros_like(tiff_data, dtype='float32')

        for band, count in enumerate(range(tiff_data.shape[0])):
            band_data = tiff_data[band, :, :]
            band_min = np.min(band_data)
            band_max = np.max(band_data)
            print("band-", count+1,"maximum-",band_max,"minimum-",band_min)
            #print(band_data)
            #epsilon is added to prevent division by zero
            normalized_band = (band_data - band_min) / (band_max - band_min + 1e-10)
            normalized_image[band, :, :] = normalized_band

        # Calculate the new width and height that are multiples of the patch size
        new_width = int(np.floor(width / patch_size)) * patch_size
        new_height = int(np.floor(height / patch_size)) * patch_size

        print("cropped dimensions:", new_height, new_width)

        input_image = np.moveaxis(normalized_image, 0, -1)

        # Crop the input_image to the new dimensions
        cropped_array = input_image[:new_height, :new_width, :]

    print("total number of nan",np.count_nonzero(np.isnan(cropped_array)))
    print("Cropped array shape:", cropped_array.shape)
    print(np.min(cropped_array), np.max(cropped_array))

    patches = []
    for i in range(0, cropped_array.shape[0], patch_size):
        for j in range(0, cropped_array.shape[1], patch_size):
            patch = cropped_array[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
    print("patches are created")
    return patches

In [12]:
def sampling(patches,indexes):
    useful_patches = []
    for number_of_masks,i in enumerate(indexes):
      print(i)
      if i < len(patches):
        useful_patches.append(patches[i])
      else:
        break
    return useful_patches, number_of_masks



In [13]:
def process_images_in_folder(datapath, patch_size, roipath,target, roishape = 'area2_square.geojson'):
    # Initialize an empty list to store the sampled patches and target_patches
    sampled_image_patches = []
    target_patches = []

    # Loop through all files in the folder
    for ps in os.listdir(datapath):

        # #print(type(filename),filename.split('.tif')[0] + '_roi.tif' )
        # print(folder_path + ps)
        # print(roipath + roishape)
        # print(ps)

        # Apply clipping
        clipped_image_path = newclipping(datapath,roipath,ps, roishape)

        # Preprocess(normalize,resize,create patches) the clipped image
        patches = preprocessing(clipped_image_path, patch_size)
        print(len(patches))

        # Sample patches from the processed image patches and create corresponding target patches
        #print(target[1])
        #print(len(target[1]))
        useful_patches, number_of_masks = sampling(patches,target[1])
        sampled_image_patches.extend(useful_patches)
        target_patches.extend(target[0][0:number_of_masks])
        print(len(sampled_image_patches),len(target_patches) )


    return sampled_image_patches,target_patches

Test the function for two images:


1.   create a folder containing two images
2.   check how many image patches and target patches are formed
3. visualise them





In [14]:
datapath = '/content/gdrive/MyDrive/exp/other images/test_folder/'
roipath = '/content/gdrive/MyDrive/exp/other images/roi_folder/'
patch_size = 256

dataset = process_images_in_folder(datapath, patch_size, roipath, target)


Performing the clip operation...

/content/gdrive/MyDrive/exp/other images/roi_folder/area2_0619_2023_8bands_composite_roi.tif /content/gdrive/MyDrive/exp/other images/test_folder/area2_0619_2023_8bands_composite.tif

Clipped raster input:  area2_0619_2023_8bands_composite_roi.tif
Checking spatial reference info

WKT format: PROJCS["WGS 84 / UTM zone 50S",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",117],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",10000000],
    UNIT["metre",1,
        AUTHORITY

In [ ]:
#apply onehot encoding to target images



In [15]:
raw_target = dataset[1]



([array([[[0.00597432, 0.02457251, 0.03490852, ..., 0.02176744,
           0.07383844, 0.36037022],
          [0.00724546, 0.0239392 , 0.0341228 , ..., 0.02018605,
           0.06024677, 0.3419586 ],
          [0.00927927, 0.02381254, 0.02592884, ..., 0.01925581,
           0.06178909, 0.3240446 ],
          ...,
          [0.00851659, 0.02811906, 0.04074531, ..., 0.024     ,
           0.07885098, 0.30483678],
          [0.00724546, 0.0290057 , 0.03468403, ..., 0.02409302,
           0.08299595, 0.2875199 ],
          [0.00953349, 0.02951235, 0.03187788, ..., 0.02344186,
           0.07326008, 0.2844347 ]],
  
         [[0.00737257, 0.02469918, 0.03367381, ..., 0.02102326,
           0.0726817 , 0.3567874 ],
          [0.00610144, 0.02419253, 0.03356157, ..., 0.01776744,
           0.06015038, 0.34375   ],
          [0.0086437 , 0.0229259 , 0.0255921 , ..., 0.01925581,
           0.06728359, 0.31956607],
          ...,
          [0.00851659, 0.02951235, 0.04231676, ..., 0.02734884,
  